In [1]:
import torch
import gpytorch as gp
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

torch.manual_seed(1)
np.random.seed(1)

In [2]:
df = pd.read_csv('C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv')

In [3]:
class MyGPModel(gp.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MyGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module  = gp.means.ZeroMean()
        self.covar_module = gp.kernels.ScaleKernel(gp.kernels.RBFKernel())

    def forward(self, x):
        mean_x  = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gp.distributions.MultivariateNormal(mean_x, covar_x)

In [4]:
for n_train in ([16, 32, 64, 128]):
    for lr in ([0.001, 0.01, 0.02, 0.05]):

        # TRAIN SET
        Tz_k   = np.array(df['room_temp']).reshape(-1,1)[0: n_train]
        Tz_k1  = np.array(df['room_temp'])[1: n_train+1]
        Tsa_k1 = np.array(df['supply_discharge_temp']).reshape(-1,1)[1: n_train+1]
        msa_k1 = np.array(df['airflow_current']).reshape(-1,1)[1: n_train+1]
        z      = ((msa_k1-400)/600)*(Tsa_k1-Tz_k)

        # train_x = np.concatenate((Tz_k, Tsa_k1, msa_k1), axis=1)
        train_x = np.concatenate((Tz_k, Tsa_k1, z), axis=1)
        train_x = torch.tensor(train_x, dtype=torch.float32)
        train_y = torch.tensor(Tz_k1, dtype=torch.float32)

        # TEST SET
        Tz_k   = np.array(df['room_temp']).reshape(-1,1)[n_train:1600]
        Tz_k1  = np.array(df['room_temp'])[n_train+1:1600+1]
        Tsa_k1 = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train+1: 1600+1]
        msa_k1 = np.array(df['airflow_current']).reshape(-1,1)[n_train+1:1600+1]
        z      = ((msa_k1-400)/600)*(Tsa_k1-Tz_k)

        # test_x = np.concatenate((Tz_k, Tsa_k1, msa_k1), axis=1)
        test_x = np.concatenate((Tz_k, Tsa_k1, z), axis=1)
        test_x = torch.tensor(test_x, dtype=torch.float32)
        test_y = torch.tensor(Tz_k1, dtype=torch.float32)

        # initialize likelihood and model
        likelihood = gp.likelihoods.GaussianLikelihood()
        model      = MyGPModel(train_x, train_y, likelihood)

        # Training
        training_iter = 200000    # number of training iteration

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)       # optimizer
        mll = gp.mlls.ExactMarginalLogLikelihood(likelihood, model)   # marginal likelihood

        R2_test = np.array([], dtype=np.float32)
        for i in range(training_iter+1):
            # TRAIN
            model.train()                   # find the hyperparameters
            likelihood.train()

            optimizer.zero_grad()           # Zero gradients from previous iteration
            output = model(train_x)         # Output from model
            loss   = -mll(output, train_y)  # Calc loss and backprop gradients

            loss.backward()
            optimizer.step()
            
            # EVAL
            model.eval()
            likelihood.eval()

            with torch.no_grad(), gp.settings.fast_pred_var():
                pred_y = likelihood(model(test_x))

            r2 = r2_score(test_y.numpy(), pred_y.mean.numpy())
            R2_test = np.append(R2_test, r2)
        
        # print(np.argmax(R2_test))
        print('n:%4d \t lr:%5.3f \t r2:%8.4f' % (n_train, lr, np.max(R2_test)))

n:  16 	 lr:0.001 	 r2:-450.0463
n:  16 	 lr:0.010 	 r2:-11.8959
n:  16 	 lr:0.020 	 r2: -4.7216
n:  16 	 lr:0.050 	 r2: -3.3482
n:  32 	 lr:0.001 	 r2:  0.9157
n:  32 	 lr:0.010 	 r2:  0.9880
n:  32 	 lr:0.020 	 r2:  0.9949
n:  32 	 lr:0.050 	 r2:  0.9975
n:  64 	 lr:0.001 	 r2:  0.9955
n:  64 	 lr:0.010 	 r2:  0.9983
n:  64 	 lr:0.020 	 r2:  0.9984
n:  64 	 lr:0.050 	 r2:  0.9984
n: 128 	 lr:0.001 	 r2:  0.9976
n: 128 	 lr:0.010 	 r2:  0.9982
n: 128 	 lr:0.020 	 r2:  0.9982
n: 128 	 lr:0.050 	 r2:  0.9982
